# ภารกิจเสริม: วิศวกรรมลักษณะข้อมูล (Feature Engineering) และการถดถอยพหุนาม (Polynomial Regression)
![](./images/C1_W2_Lab07_FeatureEngLecture_v1.png)


## เป้าหมาย
ในห้องปฏิบัติการนี้ คุณจะ:
- สำรวจการออกแบบฟีเจอร์ (feature engineering) และการถดถอยพหุนาม (polynomial regression) ซึ่งช่วยให้คุณสามารถใช้เครื่องจักรของการถดถอยเชิงเส้นเพื่อปรับให้เข้ากับฟังก์ชันที่ซับซ้อนมาก ๆ แม้กระทั่งฟังก์ชันที่ไม่เชิงเส้นมาก ๆ

## เครื่องมือ
คุณจะใช้ฟังก์ชันที่พัฒนาในแล็บก่อนหน้านี้ รวมถึง matplotlib และ NumPy

In [ ]:
# prompt: import requsts and download from this github link : https://raw.githubusercontent.com/Smith-WeStrideTH/Machine_Learning_Course/main/work/deeplearning.mplstyle

import requests

url2 = 'https://raw.githubusercontent.com/Smith-WeStrideTH/Machine_Learning_Course/main/work/lab_utils_multi.py'

response = requests.get(url2)
with open('lab_utils_multi.py', 'wb') as f:
  f.write(response.content)


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from lab_utils_multi import zscore_normalize_features, run_gradient_descent_feng
np.set_printoptions(precision=2)  # reduced display precision on numpy arrays

<a name='FeatureEng'></a>
# การออกแบบคุณลักษณะ (Feature Engineering) และการถดถอยแบบพหุนาม (Polynomial Regression)

การถดถอยเชิงเส้น (Linear Regression) แบบพื้นฐานช่วยสร้างโมเดลในรูปแบบต่อไปนี้:
$$f_{\mathbf{w},b} = w_0x_0 + w_1x_1+ ... + w_{n-1}x_{n-1} + b \tag{1}$$ 
แต่ถ้าคุณลักษณะ (features) หรือข้อมูลของคุณไม่เป็นเชิงเส้น หรือเป็นการรวมกันของคุณลักษณะหลายตัวล่ะ? ตัวอย่างเช่น ราคาบ้านอาจไม่สัมพันธ์เชิงเส้นกับพื้นที่อยู่อาศัย แต่จะมีค่าใช้จ่ายเพิ่มขึ้นอย่างรวดเร็วสำหรับบ้านที่มีขนาดเล็กมากหรือขนาดใหญ่มาก ส่งผลให้เกิดเส้นโค้งดังภาพด้านบน
เราจะใช้เครื่องมือของการถดถอยเชิงเส้นเพื่อปรับให้เข้ากับเส้นโค้งนี้ได้อย่างไร? จำไว้ว่า 'เครื่องมือ' ที่เรามีคือความสามารถในการปรับเปลี่ยนพารามิเตอร์  $\mathbf{w}$, $\mathbf{b}$ in (1)  ในสมการ (1) เพื่อ 'ปรับ' สมการให้เข้ากับข้อมูลการฝึก (training data) อย่างไรก็ตาม การปรับเปลี่ยน $\mathbf{w}$,$\mathbf{b}$  ในสมการ (1) เท่าใดก็ตาม จะไม่สามารถทำให้เข้ากับเส้นโค้งแบบไม่เชิงเส้นได้


<a name='PolynomialFeatures'></a>
## ฟีเจอร์แบบพหุนาม (Polynomial Features)

ก่อนหน้านี้ เราได้พิจารณาสถานการณ์ที่ข้อมูลไม่ใช่เชิงเส้นตรง (non-linear)
ลองใช้สิ่งที่เรารู้มาจนถึงตอนนี้เพื่อปรับโค้งแบบไม่ใช่เชิงเส้นตรงกัน
เราจะเริ่มต้นด้วยฟังก์ชันกำลังสองง่ายๆ: $y = 1+x^2$

คุณคุ้นเคยกับฟังก์ชันทั้งหมดที่เรากำลังใช้อยู่แล้ว คุณสามารถดูรายละเอียดเพิ่มเติมได้ในไฟล์  lab_utils.py [`np.c_[..]`](https://numpy.org/doc/stable/reference/generated/numpy.c_.html) เป็นฟังก์ชันใน NumPy ที่ใช้สำหรับการเชื่อมต่อ (concatenate) ข้อมูลตามแนวคอลัมน์

In [ ]:
# create target data
x = np.arange(0, 20, 1)
y = 1 + x**2
X = x.reshape(-1, 1)

model_w,model_b = run_gradient_descent_feng(X,y,iterations=1000, alpha = 1e-2)

plt.scatter(x, y, marker='x', c='r', label="Actual Value"); plt.title("no feature engineering")
plt.plot(x,X@model_w + model_b, label="Predicted Value");  plt.xlabel("X"); plt.ylabel("y"); plt.legend(); plt.show()

ดังที่คาดไว้ การใช้โมเดลเชิงเส้นตรงอาจไม่เหมาะสม สิ่งที่จำเป็นคือโมเดลที่มีรูปแบบคล้ายกับ $y= w_0x_0^2 + b$ หรือเรียกว่า โมเดลพหุนาม (polynomial feature)
เพื่อให้บรรลุเป้าหมายนี้ คุณสามารถปรับเปลี่ยน ข้อมูลอินพุต เพื่อ ออกแบบ ฟีเจอร์ที่ต้องการได้
หากคุณสลับข้อมูลต้นฉบับด้วยเวอร์ชันที่ยกกำลังสองของค่า $x$ คุณจะสามารถบรรลุ  $y= w_0x_0^2 + b$. ได้ ลองทำตามนี้ สลับ `X` เป็น  `X**2` ดังต่อไปนี้:

In [ ]:
# create target data
x = np.arange(0, 20, 1)
y = 1 + x**2

# Engineer features 
X = x**2      #<-- added engineered feature

In [ ]:
X = X.reshape(-1, 1)  #X should be a 2-D Matrix
model_w,model_b = run_gradient_descent_feng(X, y, iterations=10000, alpha = 1e-5)

plt.scatter(x, y, marker='x', c='r', label="Actual Value"); plt.title("Added x**2 feature")
plt.plot(x, np.dot(X,model_w) + model_b, label="Predicted Value"); plt.xlabel("x"); plt.ylabel("y"); plt.legend(); plt.show()

กราฟที่ใกล้เคียงกับเป้าหมายอย่างมาก
สังเกตค่าของ $w$ และ $b$ ที่พิมพ์ด้านบนของกราฟ: `w,b พบว่า gradient descent: w: [1.], b: 0.0490`. การไล่ระดับ (Gradient Descent) ได้ปรับค่าเริ่มต้นของ  $\mathbf{w},b $ ให้เป็น (1.0,0.049) ซึ่งสอดคล้องกับโมเดล $y=1*x_0^2+0.049$, ใกล้เคียงกับเป้าหมาย $y=1*x_0^2+1$. หากคุณให้มันทำงานนานขึ้น อาจจะได้ผลลัพธ์ที่ดีกว่านี้

### เลือกฟีเจอร์ที่เหมาะสมสำหรับการสร้างโมเดล
<a name='GDF'></a>
ในตัวอย่างก่อนหน้า เราทราบว่าจำเป็นต้องใช้เทอม $x^2$ อย่างไรก็ตาม ในปัญหาจริง อาจไม่ชัดเจนว่าฟีเจอร์ใดจำเป็นต้องใช้บ้าง  เราสามารถทดลองเพิ่มฟีเจอร์ต่างๆ เพื่อหาฟีเจอร์ที่เหมาะสมที่สุด

ตัวอย่าง: เราอาจลองใช้สมการต่อไปนี้แทน: $y=w_0x_0 + w_1x_1^2 + w_2x_2^3+b$ ? 

ลองรันเซลล์ถัดไปเพื่อทดสอบ

In [ ]:
# create target data
x = np.arange(0, 20, 1)
y = x**2

# engineer features .
X = np.c_[x, x**2, x**3]   #<-- added engineered feature

In [ ]:
model_w,model_b = run_gradient_descent_feng(X, y, iterations=10000, alpha=1e-7)

plt.scatter(x, y, marker='x', c='r', label="Actual Value"); plt.title("x, x**2, x**3 features")
plt.plot(x, X@model_w + model_b, label="Predicted Value"); plt.xlabel("x"); plt.ylabel("y"); plt.legend(); plt.show()

หมายเหตุเกี่ยวกับค่า  $\mathbf{w}$, `[0.08 0.54 0.03]` และ b คือ  `0.0106`.สิ่งนี้หมายความว่าโมเดลหลังจากการฟิต (fitting) หรือการฝึก (training) คือ:
$$ 0.08x + 0.54x^2 + 0.03x^3 + 0.0106 $$
การไล่ระดับ (gradient descent) ได้เน้นให้ความสำคัญกับข้อมูลที่เหมาะสมที่สุดกับข้อมูล  $x^2$ โดยเพิ่มค่า $w_1$ เมื่อเทียบกับค่าอื่นๆ  Iหากคุณเรียกใช้การไล่ระดับเป็นเวลานานมาก มันจะลดผลกระทบของเทอมอื่นๆ ต่อไป
>Gradient descent is picking the 'correct' features for us by emphasizing its associated parameter

ทบทวนแนวคิดนี้:
- ค่าน้ำหนัก (weight) ที่มีค่าน้อยกว่า หมายถึงฟีเจอร์นั้นมีความสำคัญหรือนำไปสู่การคาดการณ์ที่ถูกต้องน้อยกว่า และในกรณีสุดขั้ว เมื่อค่าน้ำหนักใกล้ศูนย์หรือเท่ากับศูนย์ ฟีเจอร์นั้นจะไม่เป็นประโยชน์ในการปรับโมเดลให้เข้ากับข้อมูล
- หลังจากการปรับโมเดล ค่าน้ำหนักที่เกี่ยวข้องกับฟีเจอร์ $x^2$ จะมีค่ามากขึ้นอย่างเห็นได้ชัดเมื่อเทียบกับค่าน้ำหนักของ  $x$ หรือ $x^3$ มีประโยชน์มากที่สุดในการปรับโมเดลให้เข้ากับข้อมูล



### มุมมองทางเลือก
ข้างต้น เราเลือกฟีเจอร์แบบพหุนาม (polynomial features) โดยพิจารณาจากความเหมาะสมกับข้อมูลเป้าหมาย วิธีการอีกอย่างหนึ่งในการคิดเกี่ยวกับเรื่องนี้คือการสังเกตว่าเรายังคงใช้การถดถอยเชิงเส้น (linear regression) หลังจากที่เราได้สร้างฟีเจอร์ใหม่แล้ว
เมื่อพิจารณาถึงเรื่องนี้แล้ว ฟีเจอร์ที่ดีที่สุดจะสัมพันธ์เชิงเส้นกับเป้าหมาย (target) ซึ่งสามารถเข้าใจได้ดีที่สุดผ่านตัวอย่างต่อไปนี้

In [ ]:
# create target data
x = np.arange(0, 20, 1)
y = x**2

# engineer features .
X = np.c_[x, x**2, x**3]   #<-- added engineered feature
X_features = ['x','x^2','x^3']

In [ ]:
fig,ax=plt.subplots(1, 3, figsize=(12, 3), sharey=True)
for i in range(len(ax)):
    ax[i].scatter(X[:,i],y)
    ax[i].set_xlabel(X_features[i])
ax[0].set_ylabel("y")
plt.show()

จากกราฟข้างต้น เราเห็นได้ชัดว่าคุณสมบัติ $x^2$ เมื่อจับคู่กับค่าเป้าหมาย $y$ มีความสัมพันธ์เชิงเส้น (linear relationship)

### มาตรการปรับขนาด (Scaling features)
ตามที่ได้กล่าวไปในแล็บก่อนหน้านี้ หากชุดข้อมูลของคุณมีคุณลักษณะ (features) ที่มีสเกลแตกต่างกันอย่างมาก ควรใช้เทคนิคการปรับขนาดคุณลักษณะ (feature scaling) เพื่อเร่งการไล่ระดับ (gradient descent)
ในตัวอย่างข้างต้น มีตัวแปร $x$, $x^2$ และ $x^3$ ซึ่งจะมีสเกลที่แตกต่างกันอย่างมากตามธรรมชาติ มาลองใช้การแปลง Z-score เพื่อปรับขนาดตัวอย่างของเราให้เหมาะสมกัน

In [ ]:
# create target data
x = np.arange(0,20,1)
X = np.c_[x, x**2, x**3]
print(f"Peak to Peak range by column in Raw        X:{np.ptp(X,axis=0)}")

# add mean_normalization 
X = zscore_normalize_features(X)     
print(f"Peak to Peak range by column in Normalized X:{np.ptp(X,axis=0)}")

ตอนนี้เราสามารถลองใช้ค่า alpha ที่เข้มข้นกว่านี้:

In [ ]:
x = np.arange(0,20,1)
y = x**2

X = np.c_[x, x**2, x**3]
X = zscore_normalize_features(X) 

model_w, model_b = run_gradient_descent_feng(X, y, iterations=100000, alpha=1e-1)

plt.scatter(x, y, marker='x', c='r', label="Actual Value"); plt.title("Normalized x x**2, x**3 feature")
plt.plot(x,X@model_w + model_b, label="Predicted Value"); plt.xlabel("x"); plt.ylabel("y"); plt.legend(); plt.show()

การปรับสเกลฟีเจอร์ช่วยให้การไล่ระดับบรรจบกันได้เร็วขึ้น
โปรดสังเกตค่าของ  $\mathbf{w}$ อีกครั้ง พจน์ $w_1$ ซึ่งเป็นพจน์ $x^2$  มีความสำคัญมากที่สุด การไล่ระดับได้ลดพจน์  $x^3$ ลงไปเกือบหมดแล้ว

### ฟังก์ชันที่ซับซ้อน (Complex Functions)
ด้วยการปรับแต่งคุณสมบัติ (feature engineering) สามารถสร้างแบบจำลองแม้แต่ฟังก์ชันที่ซับซ้อนได้:

In [ ]:
x = np.arange(0,20,1)
y = np.cos(x/2)

X = np.c_[x, x**2, x**3,x**4, x**5, x**6, x**7, x**8, x**9, x**10, x**11, x**12, x**13]
X = zscore_normalize_features(X) 

model_w,model_b = run_gradient_descent_feng(X, y, iterations=1000000, alpha = 1e-1)

plt.scatter(x, y, marker='x', c='r', label="Actual Value"); plt.title("Normalized x x**2, x**3 feature")
plt.plot(x,X@model_w + model_b, label="Predicted Value"); plt.xlabel("x"); plt.ylabel("y"); plt.legend(); plt.show()


## ยินดีด้วย!
ใน Lab นี้ คุณได้เรียนรู้:

- วิธีที่การถดถอยเชิงเส้น (linear regression) - สามารถสร้างแบบจำลองฟังก์ชันที่ซับซ้อนแม้กระทั่งแบบที่ไม่เชิงเส้นอย่างมาก โดยใช้การวิศวกรรมลักษณะ (feature engineering)ความสำคัญของการปรับขนาดลักษณะ (feature scaling) เมื่อทำการวิศวกรรมลักษณะ